# Advent of Code 2021
## [Day 16: Packet Decoder](https://adventofcode.com/2021/day/16)

#### Load Data

In [1]:
with open('input.txt') as f:
    input_data = f.read().strip()
input_data

'A059141803C0008447E897180401F82F1E60D80021D11A3DC3F300470015786935BED80A5DB5002F69B4298A60FE73BE41968F48080328D00427BCD339CC7F431253838CCEFF4A943803D251B924EC283F16D400C9CDB3180213D2D542EC01092D77381A98DA89801D241705C80180960E93469801400F0A6CEA7617318732B08C67DA48C27551C00F972830052800B08550A277416401A5C913D0043D2CD125AC4B1DB50E0802059552912E9676931530046C0141007E3D4698E20008744D89509677DBF5759F38CDC594401093FC67BACDCE66B3C87380553E7127B88ECACAD96D98F8AC9E570C015C00B8E4E33AD33632938CEB4CD8C67890C01083B800E5CBDAB2BDDF65814C01299D7E34842E85801224D52DF9824D52DF981C4630047401400042E144698B2200C4328731CA6F9CBCA5FBB798021259B7B3BBC912803879CD67F6F5F78BB9CD6A77D42F1223005B8037600042E25C158FE0008747E8F50B276116C9A2730046801F29BC854A6BF4C65F64EB58DF77C018009D640086C318870A0C01D88105A0B9803310E2045C8CF3F4E7D7880484D0040001098B51DA0980021F17A3047899585004E79CE4ABD503005E610271ED4018899234B64F64588C0129EEDFD2EFBA75E0084CC659AF3457317069A509B97FB3531003254D080557A00CC8401F8791DA13080391EA39C739EFE

### Part 1

In [2]:
a = int(input_data[0], 16)

In [3]:
f"{int('2', 16):04b}"

'0010'

In [4]:
example_literal = 'D2FE28'

In [5]:
def hex_to_bin(s):
    binary = ''
    for c in s:
        nibble = f"{int(c, base=16):04b}"
        binary += nibble
    return binary

packet = hex_to_bin(example_literal)
packet

'110100101111111000101000'

In [6]:
class BITS(object):
    def __init__(self, input_str, base=16):
        self.cursor = 0
        if base == 2:
            self.bits = input_str
        else:
            self.bits = ''
            for char in input_str:
                nibble = f"{int(char, base):04b}"
                self.bits += nibble
        
    def __repr__(self):
        return f"<BITS {self.bits[self.cursor:]}>"
    
    @property
    def remaining(self):
        return len(self.bits) - self.cursor
        
    def get_str(self, n):
        value = self.bits[self.cursor:self.cursor+n]
        self.cursor += n
        return value
    
    def get_sub_bits(self, n):
        sub_bits = self.get_str(n)
        return BITS(sub_bits, base=2)
    
    def get(self, n):
        return int(self.get_str(n), base=2)

BITS(example_literal)

<BITS 110100101111111000101000>

In [7]:
b = BITS(example_literal)
b.remaining, b.get(3), b.remaining

(24, 6, 21)

In [8]:
example_operator = '38006F45291200'
packet = hex_to_bin(example_operator)
packet

'00111000000000000110111101000101001010010001001000000000'

In [14]:
from functools import reduce

class Packet(object):
    def __init__(self, bits):
        self.packet_version = bits.get(3)
        self.packet_type = bits.get(3)
        self.sub_packets = []
        
        # literal packet type
        if self.packet_type == 4:
            literal_value = 0
            terminate = False
            while not terminate:
                terminate = (bits.get(1) == 0)
                nibble = bits.get(4)
                literal_value = literal_value * 16 + nibble
            self.value = literal_value
            return

        # operator packet type
        length_type = bits.get(1)
        if length_type == 0:
            num_sub_packet_bits = bits.get(15)
            sub_packets_bits = bits.get_sub_bits(num_sub_packet_bits)
            while sub_packets_bits.remaining > 7:
                self.sub_packets.append(Packet(sub_packets_bits))
        else:
            num_sub_packets = bits.get(11)
            for _ in range(num_sub_packets):
                self.sub_packets.append(Packet(bits))
                
    def __repr__(self):
        return f"<Packet v{self.packet_version} type={self.packet_type} len={len(self.sub_packets)}>"

    def total_version(self):
        return self.packet_version + sum(p.total_version() for p in self.sub_packets)
    
    def evaluate(self):
        if self.packet_type == 4:
            return self.value
        values = [p.evaluate() for p in self.sub_packets]
        if self.packet_type == 0:
            return sum(values)
        elif self.packet_type == 1:
            return reduce((lambda x, y: x * y), values)
        elif self.packet_type == 2:
            return min(values)
        elif self.packet_type == 3:
            return max(values)
        elif self.packet_type == 5:
            return int(values[0] > values[1])
        elif self.packet_type == 6:
            return int(values[0] < values[1])
        elif self.packet_type == 7:
            return int(values[0] == values[1])
        
Packet(BITS(example_operator))

<Packet v1 type=6 len=2>

In [17]:
print(Packet(BITS(input_data)).sub_packets)

[<Packet v0 type=1 len=3>, <Packet v7 type=1 len=2>, <Packet v5 type=2 len=4>, <Packet v0 type=1 len=2>, <Packet v1 type=4 len=0>, <Packet v0 type=3 len=3>, <Packet v3 type=2 len=1>, <Packet v6 type=1 len=2>, <Packet v1 type=1 len=1>, <Packet v3 type=4 len=0>, <Packet v7 type=1 len=3>, <Packet v1 type=2 len=5>, <Packet v4 type=3 len=1>, <Packet v0 type=1 len=2>, <Packet v3 type=1 len=2>, <Packet v0 type=1 len=4>, <Packet v5 type=1 len=2>, <Packet v1 type=4 len=0>, <Packet v5 type=0 len=2>, <Packet v6 type=3 len=5>, <Packet v4 type=1 len=2>, <Packet v4 type=1 len=2>, <Packet v5 type=1 len=2>, <Packet v3 type=4 len=0>, <Packet v6 type=1 len=2>, <Packet v7 type=1 len=2>, <Packet v2 type=1 len=2>, <Packet v4 type=4 len=0>, <Packet v7 type=1 len=2>, <Packet v3 type=0 len=3>, <Packet v4 type=4 len=0>, <Packet v7 type=4 len=0>, <Packet v6 type=1 len=2>, <Packet v0 type=2 len=2>, <Packet v0 type=1 len=2>, <Packet v3 type=2 len=3>, <Packet v2 type=1 len=2>, <Packet v6 type=4 len=0>, <Packet v0 

In [10]:
example_operator_2 = 'EE00D40C823060'
Packet(BITS(example_operator_2))

<Packet v7 type=3 len=3>

In [11]:
Packet(BITS(example_operator))

<Packet v1 type=6 len=2>

#### Part 1 Answer
Decode the structure of your hexadecimal-encoded BITS transmission;  
**what do you get if you add up the version numbers in all packets?**

In [12]:
Packet(BITS(input_data)).total_version()

860

### Part 2

#### Part 2 Answer
**What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?**

In [13]:
Packet(BITS(input_data)).evaluate()

470949537659